<!-- <center><center><img src="https://raw.githubusercontent.com/makaires77/fioce/master/assets/logo_fioce.png" 
style="height:150px" alt="Logo_Fiocruz"></center></center> -->

<center><center><img src="https://user-images.githubusercontent.com/61051085/81343928-3ce9d500-908c-11ea-9850-0210b4e94ba0.jpg" 
style="height:150px" alt="Logo_Unifor"></center></center>

## <center>Gerar o Modelo de Grafo no Neo4j<br /></center>

    Antonio Marcos Aires Barbosa (Fiocruz Ceará)

**Introdução**

Nesta etapa do trabalho populamos o banco de dados de grafo com os dados dos currículos previamente extraídos do Lattes dos servidores da unidade Fiocruz Ceará.

# Instanciar a classe e estabelecer conexão com Neo4j

In [1]:
import os
import sys

# Base do diretório do repositório
base_repo_dir = None
if 'win' in sys.platform:
    base_repo_dir = 'C:\\Users\\marcos.aires\\gml_classifier-1'  # Caminho base no Windows
else:
    base_repo_dir = '/home/mak/gml_classifier-1'  # Caminho base em Linux

# Construir os caminhos usando os.path.join
folder_utils = os.path.join(base_repo_dir, 'utils')
folder_domain = os.path.join(base_repo_dir, 'source', 'domain')
folder_data_input = os.path.join(base_repo_dir, 'data', 'input')
folder_data_output = os.path.join(base_repo_dir, 'data', 'output')

# Adicionar pastas locais ao sys.path para permitir importação de pacotes
sys.path.append(folder_utils)
sys.path.append(folder_domain)
from bert_embeedings_to_neo4j import BertEmbeddingsToNeo4j, Neo4jConnector
from experiment_monitor import ExperimentMonitor
from articles_counter import ArticlesCounter
from json_fle_manager import JSONFileManager as jfm
from experiment_profiler import TimeProfiler
from lda_extractor import LDAExtractor

# Para o caso de folder_data_prod, que parece ser exclusivo para ambientes Unix
folder_data_prod = os.path.join(base_repo_dir, 'data') if not 'win' in sys.platform else None

print(f" Caminho base do repositório: {base_repo_dir}")
print(f"Arquivos de entrada de dados: {jfm.list_json_files(folder_data_input)}")

# Definir arquivo dados brutos a processar e gerar dataset
profiler = TimeProfiler()
monitor = ExperimentMonitor(base_repo_dir, profiler)
filename = 'dict_list_fioce.json'
dict_list = monitor.load_from_json(folder_data_input,filename)
print(f"{' '*30}{len(dict_list)} currículos carregados arquivo: '{filename}'")

if monitor.is_gpu_available():
    print("\nGPU disponível para execução de código.")
else:
    print("\nNão foi detectada nenhuma GPU configurada corretamente no ambiente.")

atualizador = ArticlesCounter(dict_list)
dtf_atualizado = atualizador.extrair_data_atualizacao(dict_list)
# dtf_atualizado

[nltk_data] Downloading package stopwords to /home/mak/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/mak/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


 Caminho base do repositório: /home/mak/gml_classifier-1
Arquivos de entrada de dados: ['normalized_dict_list.json', 'dict_list_fioce.json']
                              34 currículos carregados arquivo: 'dict_list_fioce.json'

GPU disponível para execução de código.


## Recuperar os dados dos artigos persistidos

In [2]:
uri = "bolt://localhost:7687"
username = "neo4j"
password = "password"

neo4j_conn = Neo4jConnector(uri, username, password)
bert_to_neo4j = BertEmbeddingsToNeo4j(uri, username, password)

articles = neo4j_conn.extract_article_data()
len(articles)

804

In [7]:
# for i in articles:
#     print(i.get('title'))

In [8]:
bert_to_neo4j.process_and_update_embeddings(articles)
neo4j_conn.close()
bert_to_neo4j.close()

AttributeError: 'list' object has no attribute 'extract_article_data'

## Ler dicionários e criar nós de Pessoas e Artigos no Neo4j

In [ ]:
# Lê lista de dicionários de um arquivo JSON local
folder='/home/mak/gml_classifier-1/data/input/'
json_file='dict_list_fioce.json'
with open(folder + json_file, 'r') as file:
    json_data = json.load(file)
    
curriculum_dict_list = bert_to_neo4j.load_data_from_json(folder + json_file)

# Persiste pesquisadores no banco de dados
results = bert_to_neo4j.process_researchers(curriculum_dict_list)
bert_to_neo4j.close()

print(results)

## Criar relacionamentos de equipes (nós do tipo Team)

In [ ]:
bert_to_neo4j.process_relationships_from_file()  # Usando valores padrão para ingerir dados de pessoal
# bert_to_neo4j.process_relationships_from_file('/caminho/personalizado', 'arquivo_alternativo.xls')

## Atribuir nomes persistidos a Equipes

In [ ]:
neo4j_connector = Neo4jConnector(uri, username, password)

# Cria uma instância da DataEntryInterface
data_entry_interface = DataEntryInterface(neo4j_connector)

# Inicia o processo de associação de equipes usando o arquivo CSV
data_entry_interface.start_process()

# Exibe o widget para associações adicionais
# Isso será útil para associar pessoas que não foram incluídas no processo CSV
data_entry_interface.display_unassociated_people_widget()

## Corrigir alguma associação indevida ou que mudou

In [ ]:
data_entry_interface = DataEntryInterface(neo4j_connector)
data_entry_interface.display_and_edit_existing_associations()

## Caso exista ainda pessoa não associada aparecerá aqui

In [ ]:
from bert_embeedings_to_neo4j import Neo4jConnector

data_interface = DataEntryInterface(neo4j_connector)
data_interface.display_unassociated_people_widget()

## Salvar associações Pessoa - Equipe em arquivo *.csv

In [ ]:
neo4j_connector.save_associations_to_csv()

## Caso exista um arquivo de associação pode ser carregado aqui

In [ ]:
# csv_file_path = "/home/mak/gml_classifier-1/data/input/relations_person_team.csv"
# neo4j_connector.associate_people_to_teams_from_csv(csv_file_path)

## Criar uma Organização com base em uma lista de Times

In [ ]:
# Lista de nomes de temas a serem conectados
team_names = ['Biotecnologia', 'Saúde da Família', 'Saúde e Ambiente', 'Saúde Digital']

# Chamada do método para criar a organização e conectar os temas
bert_to_neo4j.create_organization_and_connect_teams("Fiocruz Ceará", "Fiocruz_CE", team_names)

## <b>Gerar os embeedings de artigos</b>

In [ ]:
!nvcc --version

In [ ]:
!nvidia-smi

In [ ]:
import torch
print(torch.__version__)

In [ ]:
print(torch.version.cuda)

In [ ]:
import torch
torch.cuda.empty_cache() # Limpa os caches da memória

In [ ]:
import sys

# Adicionar o caminho até das pastas locais ao sys.path
from bert_embeedings_to_neo4j import Ber

uri = "bolt://localhost:7687"
username = "neo4j"
password = "password"

# Exemplo de uso
neo4j_conn = Neo4jConnector(uri, username, password)
bert_embedder = BertEmbeddingsToNeo4j(uri, username, password)

bert_embedder.execute_all_embeddings_update(neo4j_conn)

neo4j_conn.close()
bert_embedder.close()

## Criar relacionamentos CNPq com dados preenchidos no Lattes

In [ ]:
bert_to_neo4j.create_cnpq_relationships()

# Deletar todos os nós de um tipo

In [ ]:
bert_to_neo4j.delete_all_nodes_of_type("Article")

In [ ]:
bert_to_neo4j.delete_all_nodes_of_type("Organization")

In [ ]:
bert_to_neo4j.delete_all_nodes_of_type("Team")

In [ ]:
bert_to_neo4j.delete_all_nodes_of_type("Person")

## Treinar um modelo LDA de separação de tópicos

In [ ]:
# Definindo o diretório base para a localização dos arquivos e parâmetros do modelo
num_topics = 8
passes = 15
random_state = 100
model_name = 'bert-base-multilingual-cased'

# Criar instância do LDAExtractor
lda_extractor = LDAExtractor(base_repo_dir, num_topics, passes, random_state, model_name)

# Nome do arquivo JSON com os dados dos documentos
json_data_filename = 'output_py_gpu_singlethread.json'

In [ ]:
## Visualizar títulos extraídos
# lda_extractor.extract_text_from_json(json_data_filename)

In [ ]:
# Chamar o método para treinar o modelo LDA
lda_model, dictionary, corpus = lda_extractor.train_lda_model(json_data_filename)

# O modelo LDA, dicionário e corpus agora estão disponíveis para uso posterior
# Exemplo: lda_model.show_topics(), dictionary.token2id, etc.

In [ ]:
lda_model.show_topics()

In [ ]:
import pickle

# Supondo que 'lda_model' seja o seu modelo LDA treinado
filepath = os.path.join(folder_data_output,'lda_model.pkl')
with open(filepath, 'wb') as file:
    pickle.dump(lda_model, file)

with open(filepath, 'rb') as file:
    loaded_lda_model = pickle.load(file)

In [ ]:
# Para usar um modelo LDA existente
lda_model_filepath = os.path.join(folder_data_output,'lda_model.pkl')
dic_gensim_filepath = os.path.join(folder_data_output,'dictionary.gensim')

# Carregar o modelo e o dicionário usando o método load_existing_model
lda_extractor.load_existing_model(lda_model_filepath, dic_gensim_filepath)

# Classificar um título
titulo = "SARS-CoV-2 genomic surveillance in Rondônia, Brazilian Western Amazon"

topico, probabilidade = lda_extractor.classify_title_to_topic(titulo)
print(f"Título pertence ao tópico {topico} com probabilidade {probabilidade:.2%}")

In [ ]:
# Para usar um modelo LDA existente
lda_model_filepath = os.path.join(folder_data_output,'lda_model.pkl')
dic_gensim_filepath = os.path.join(folder_data_output,'dictionary.gensim')

# Carregar o modelo e o dicionário usando o método load_existing_model
lda_extractor.load_existing_model(lda_model_filepath, dic_gensim_filepath)

# Agora você pode chamar plot_wordcloud sem argumentos
# lda_extractor.plot_wordcloud()

# lda_extractor.plot_wordcloud_plotly(topic_id=0)  # Visualizar a nuvem de palavras do primeiro tópico